In [ ]:
import sys
sys.path.append("..")
sys.path.append("../model")
from trajcl import TrajCL
from config import Config

In [ ]:
conf = Config()

conf.dataset = 'usa_large_cell'
conf.post_value_updates()

In [ ]:
model = TrajCL()

In [ ]:
model

In [ ]:
# from ah_databricks_data_loader import DatabricksDataLoader

# # Instantiate the DatabricksDataLoader.
# ddl = DatabricksDataLoader()

# # Load data.
# test_df = ddl.load_as_spark(schema="datascience_scratchpad", table="nyc_traj_data_v18")

In [ ]:
# test_df.count()

In [ ]:
# test_df.show(5)

In [ ]:
# test_df.write.mode("overwrite").parquet("/home/sagemaker-user/TrajCL/data/nyc/nyc_data_v18")

In [ ]:
# test_df.count()

In [ ]:
from glob import glob
import pandas as pd
parquet_files = glob("/home/sagemaker-user/TrajCL/data/nyc/nyc_data_v18/*.parquet")
total_count = 0
df_list = []
for file in parquet_files:
    df = pd.read_parquet(file)
    df_list.append(df)
    total_count += len(df)


print(f"Total records in repartitioned files: {total_count}")

In [ ]:
df_list[0].head()

In [ ]:
# import pandas as pd
# test_df = pd.read_parquet("/home/sagemaker-user/TrajCL/data/parquet_files/test/nyc_df_v3_with_time/traj_test_df_v3_with_ts.parquet")

In [ ]:
# userids = test_df['userid'].unique()

In [ ]:
import torch
device = torch.device("cuda:0")
checkpoint_file = "/home/sagemaker-user/TrajCL/exp/nyc_hier_time_weighted_v2/ep2_batch10000"
checkpoint = torch.load(checkpoint_file)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

In [ ]:
from utils.traj import *
import pickle
from utils.cellspace import *

from torch.nn.utils.rnn import pad_sequence
embs_parent = pickle.load(open(Config.dataset_embs_file_parent, 'rb')).to('cpu').detach() # tensor
embs_child = pickle.load(open(Config.dataset_embs_file_child, 'rb')).to('cpu').detach() # tensor
cellspace_parent = pickle.load(open(Config.dataset_cell_file_parent, 'rb'))
cellspace_child = pickle.load(open(Config.dataset_cell_file_child, 'rb'))
hier_cellspace = HirearchicalCellSpace(cellspace_parent, cellspace_child)

max_batch_size = 512
def infer_batch(traj, time_indices):
    traj_cell_parent, traj_cell_child, traj_p, traj_timedelta = zip(*[merc2cell2(l,t, hier_cellspace) for l,t in zip(traj, time_indices)])
    # print(traj_cell)
    traj_emb_p = [torch.tensor(generate_spatial_features(t, hier_cellspace)) for t in traj_p]
    traj_emb_p = pad_sequence(traj_emb_p, batch_first = False).to(device)
    traj_emb_cell_parent = [embs_parent[list(t)] for t in traj_cell_parent]
    traj_emb_cell_child = [embs_child[list(t)] for t in traj_cell_child]
    traj_emb_cell = [a + b for a, b in zip(traj_emb_cell_parent, traj_emb_cell_child)]
    traj_emb_cell = pad_sequence(traj_emb_cell, batch_first = False).to(device)
    traj_len = torch.tensor(list(map(len, traj_p)), dtype = torch.long, device = device)
    traj_timedelta = pad_sequence([torch.log(torch.tensor(t)) for t in traj_timedelta], batch_first=False, padding_value=0).to(Config.device)
    # print(traj_emb_cell, traj_emb_p, traj_len)
    traj_embs = model.interpret(traj_emb_cell.float(), traj_emb_p.float(), traj_len, traj_timedelta)
    return traj_embs, traj_cell_parent, traj_cell_child , traj_p, traj_timedelta

def infer(traj, time_indices):
    if len(traj)> max_batch_size:
        traj_embs = []
        for i in range(0, len(traj), max_batch_size):
            traj_batch = traj[i:i+max_batch_size]
            time_indices_batch = time_indices[i:i+max_batch_size] 
            traj_embs.append(infer_batch(traj_batch, time_indices_batch))
        return torch.cat(traj_embs, dim=0)
    else:
        return infer_batch(traj, time_indices)



In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
df_list[0]['time_index_list'].values[0]

In [ ]:
unique_userids = pd.concat([df['userid'] for df in df_list]).unique()
print(f"Total unique userids: {len(unique_userids)}")

unique_employernames = pd.concat([df['employername'] for df in df_list]).unique()
print(f"Total unique employernames: {len(unique_employernames)}")

In [ ]:
def get_data_for_userid(userid):
    df_user = pd.concat([df[df['userid']==userid] for df in df_list]).reset_index(drop=True)
    return df_user

def get_data_for_employername(df_user, employername):
    df_emp = df_user[df_user['employername']==employername].reset_index(drop=True)
    return df_emp


def get_data_for_partition(df_emp, partition):
    df_part = df_emp[df_emp['partition_id']==partition].reset_index(drop=True)
    df_part = df_part[~df_part['weekday'].isin([5, 6])].reset_index(drop=True) # filter out weekends
    df_part = df_part[df_part['pck_amt']>0].reset_index(drop=True) # filter out zero paycheck amount
    return df_part


def get_traj_and_time_data(df_part):
    traj = df_part['merc_seq'].values
    time_indices = df_part['timestamps'].values
    return traj, time_indices

In [ ]:
# def get_min_max_xy(traj):
#     min_x = min([p[0] for p in traj])
#     max_x = max([p[0] for p in traj])
#     min_y = min([p[1] for p in traj])
#     max_y = max([p[1] for p in traj])
#     return min_x, max_x, min_y, max_y

# def calculate_dx_dy(k, target_min_x, target_min_y, min_x, max_x, min_y, max_y):
#     dx = target_min_x+ (k - max_x - min_x) / 2
#     dy = target_min_y+ (k - max_y - min_y) / 2
#     return dx, dy

# def transform_traj(traj, dx, dy):
#     new_traj = [[p[0] + dx, p[1] + dy] for p in traj]
#     return new_traj

# def normalize(trajs, cellspace):
#     # trajs: list of [[lon, lat], [,], ...]

#     # 1. augment the input traj in order to form 2 augmented traj views
#     # 2. convert augmented trajs to the trajs based on mercator space by cells
#     # 3. read cell embeddings and form batch tensors (sort, pad)

#     trajs1, trajs2 = [], []
#     time_indices1, time_indices2 = [], []
#     final_min_x = 1000000000
#     final_min_y = 1000000000
#     final_max_x = -1000000000
#     final_max_y = -1000000000
#     max_len_meters = min(cellspace.x_max-cellspace.x_min, cellspace.y_max-cellspace.y_min)
#     indices_to_omit = []
#     for i, l in enumerate(trajs):
#         min_x, max_x, min_y, max_y = get_min_max_xy(l)
#         # if final_max_x - min_x> max_len_meters or final_max_y-min_y> max_len_meters or max_x - final_min_x> max_len_meters or max_y - final_min_y> max_len_meters or max_x-min_x> max_len_meters or max_y-min_y> max_len_meters:
#         #     indices_to_omit.append(i)
#         #     continue
#         final_min_x = min(final_min_x, min_x)
#         final_min_y = min(final_min_y, min_y)
#         final_max_x = max(final_max_x, max_x)
#         final_max_y = max(final_max_y, max_y)
    
#     if max(final_max_x-final_min_x, final_max_y-final_min_y)>max_len_meters:
#         # print(max(final_max_x-final_min_x, final_max_y-final_min_y), max_len_meters)
#         return None
#     dx, dy = calculate_dx_dy(max_len_meters, cellspace.x_min, cellspace.y_min, final_min_x, final_max_x, final_min_y, final_max_y)
#     # print(final_min_x, final_max_x, final_min_y, final_max_y)
#     # print(dx, dy)
#     for i,l in enumerate(trajs):
#         if i in indices_to_omit:
#             continue
#         new_l = transform_traj(l, dx, dy)
#         trajs1.append(new_l)
#     return trajs1

In [ ]:
def apply_dbscan(embs, target_min_similarity=0.85):
    from sklearn.cluster import DBSCAN     # require >= 0.9 cosine similarity
    eps = 1.0 - target_min_similarity    # cosine distance threshold
    n_embs = embs.shape[0]
    db = DBSCAN(eps=eps, min_samples=max(int(n_embs*0.3), 5), metric="cosine", n_jobs=-1).fit(embs)
    return db

In [ ]:
from tqdm import tqdm
model.eval()
output_dict = {}

# 0: moving, 1: static, 2: not enough data
def get_gt_and_pred_label(userid):
    count = 0
    user_data = get_data_for_userid(userid)
    employers = user_data['employername'].unique()
    for employer in employers:
        emp_data = get_data_for_employername(user_data, employer)
        partitions = emp_data['partition_id'].unique()
        for partition in partitions:
            # print(userid, employer, partition)
            part_data = get_data_for_partition(emp_data, partition)
            if len(part_data)<15:
                continue
            else:
                # print(userid, employer, partition)
                # print(len(part_data))
                traj, time_indices = get_traj_and_time_data(part_data)
                # print(traj)
                embs, traj_cell_parent, traj_cell_child , traj_p, traj_timedelta = infer(traj, time_indices)
                db_scan = apply_dbscan(embs.detach().cpu().numpy(), target_min_similarity=0.85)
                labels = db_scan.labels_                      # shape: (n_samples,)
                cluster_ids = [c for c in np.unique(labels) if c != -1]
                if len(cluster_ids)>0:
                    pred_label = 1
                else:
                    pred_label = 0
                output_dict[(userid, employer, partition)] = pred_label

    return count
    # if sum(test_data['paycheck_amount'].values) > 0:
    #     gt_label = 1
    # else:
    #     gt_label = 0
    # pred_label = 0
    # for i in range(len(test_embs)):
    #     test_vector = test_embs[i].unsqueeze(0)
    #     similarity = cosine_similarity(test_vector.numpy(), train_embs)[0]
    #     top_3_indices = np.argsort(similarity)[-3:][::-1]
    #     # print(i, top_3_indices)
    #     similarity = similarity[top_3_indices]
    #     # print(f"User: {userid}, Test Trajectory {test_data['traj_id'].values[i]}, Top 3 Train Trajectories: {train_data['traj_id'].values[top_3_indices]}, similarity: {similarity}, PCK Amount: {train_data['paycheck_amount'].values[top_3_indices]}")
    #     for sim, idx in zip(similarity, top_3_indices):
    #         if sim>0.85 and train_data['paycheck_amount'].values[idx]>0:
    #             pred_label = 1
    #             break
    # return gt_label, pred_label
count=0
for i, userid in tqdm(enumerate(unique_userids)):
    count+=get_gt_and_pred_label(userid)
    if i>1000:
        break

        


In [ ]:
# save output_dict
pickle.dump(output_dict, open("/home/sagemaker-user/TrajCL/exp/v2.1/nyc_test_hyp1_output_dict.pkl", 'wb'))

In [ ]:
output_dict

In [ ]:
count_dict = {}
for k,v in output_dict.items():
    if v in count_dict:
        count_dict[v]+=1
    else:
        count_dict[v]=1
count_dict

In [ ]:
def get_data_userid_employer_partition(userid, employer, partition):
    df_user = pd.concat([df[df['userid']==userid] for df in df_list]).reset_index(drop=True)
    df_emp = df_user[df_user['employername']==employer].reset_index(drop=True)
    df_part = df_emp[df_emp['partition_id']==partition].reset_index(drop=True)
    df_part_fil = df_part[~df_part['weekday'].isin([5, 6])].reset_index(drop=True) # filter out weekends
    df_part_fil = df_part_fil[df_part_fil['pck_amt']>0].reset_index(drop=True) # filter out zero paycheck amount
    return df_part_fil

In [ ]:
output_dict[(2523559, 'united ground express', 0)]

In [ ]:
df_part = get_data_userid_employer_partition(2523559, 'united ground express', 0)
df_part.sort_values('traj_date')

In [ ]:
import numpy as np

def cosine_similarity_matrix(X, eps=1e-8, dtype=np.float32):
    X = np.asarray(X, dtype=dtype)
    norms = np.linalg.norm(X, axis=1, keepdims=True)
    Xn = X / np.maximum(norms, eps)
    return Xn @ Xn.T

In [ ]:
def get_cluster(part_df):
    traj , time_indices= get_traj_and_time_data(part_df)
    embs, _, _, _, _ = infer(traj,time_indices)
    db_scan = apply_dbscan(embs.detach().cpu().numpy(), target_min_similarity=0.85)
    labels = db_scan.labels_                      # shape: (n_samples,)
    cluster_ids = [c for c in np.unique(labels) if c != -1]
    for i in part_df.index:
        print(labels[i], part_df.iloc[i]['traj_date'])
    sim = cosine_similarity_matrix(embs.detach().cpu().numpy())
    sim_pairs = []
    for i in range(sim.shape[0]):
        for j in range(i+1, sim.shape[1]):
            if sim[i,j]>0.85:
                sim_pairs.append((i, j, sim[i,j]))
    return sim_pairs

In [ ]:
def get_similarity(part_df):
    traj_dates = list(part_df['traj_date'])
    traj , time_indices= get_traj_and_time_data(part_df)
    # traj = normalize(traj, cellspace)
    # print(traj)
    embs, parent_cell, child_cell, traj_p, time_delta = infer(traj,time_indices)
    similarity = cosine_similarity_matrix(embs.detach().cpu().numpy())


    return similarity, parent_cell, child_cell, traj_p, time_delta

In [ ]:
import datetime

In [ ]:
get_cluster(df_part)

In [ ]:
df_fil= df_part[df_part['traj_date'].isin([datetime.date(2024,9,19), datetime.date(2024,9,13)])]
sim, parent_cell, child_cell, traj_p, time_delta = get_similarity(df_fil)

In [ ]:
sim

In [ ]:
time_delta = time_delta.cpu().numpy()

In [ ]:

sorted_indices_0 = list(np.argsort(time_delta[:,0]))[::-1]

cell_parent_fil_0 = np.array(parent_cell[0])[sorted_indices_0[:10]]
cell_fil_0 = np.array(child_cell[0])[sorted_indices_0[:10]]
timestamps_0 = time_delta[sorted_indices_0[:10],0]

sorted_indices_1 = list(np.argsort(time_delta[:,1]))[::-1]
cell_parent_fil_1 = np.array(parent_cell[1])[sorted_indices_1[:10]]
cell_fil_1 = np.array(child_cell[1])[sorted_indices_1[:10]]
timestamps_1 = time_delta[sorted_indices_1[:10],1]


for i in range(len(cell_fil_0)):
    print(cell_parent_fil_0[i], cell_fil_0[i], timestamps_0[i], math.exp(timestamps_0[i]), cell_parent_fil_1[i], cell_fil_1[i], timestamps_1[i], math.exp(timestamps_1[i]))

In [ ]:
print(sorted_indices_0)
print(sorted_indices_1)